In [1]:
import cv2
import socket
import time

# Load the face detection model
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Open a video strea
cap = cv2.VideoCapture('192.168.1.134:81')

# Define the Wi-Fi sending function
def send_number_over_wifi():
    HOST = '192.168.1.134'  # Replace with the IP address of the device receiving the number
    PORT = 80  # Replace with the port number of the receiving device
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.connect((HOST, PORT)) #ip adres en poort invullen (udp of tcp vermoed ik)
        s.sendall(b'300')

# Loop through the frames in the video stream
while True:
    # Read the next frame from the video stream
    ret, frame = cap.read()
    time.sleep(1)
    print(cap.read())
    if frame is None:
        continue
    
    
    # Apply the face detection model to the frame
    faces = face_cascade.detectMultiScale(frame, scaleFactor=1.1, minNeighbors=5)

    # Draw bounding boxes around the detected faces
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        # Send the number 300 over Wi-Fi
        send_number_over_wifi()

    # Display the output
    cv2.imshow('frame', frame)

    # Exit if the 'q' key is pressed
    if cv2.waitKey(1) == ord('q'):
        break

# Release the video stream and close the window
cap.release()
cv2.destroyAllWindows()

(False, None)
(False, None)


KeyboardInterrupt: 

In [28]:
import cv2
import socket
import time
import numpy as np
from everywhereml.data import ImageDataset
from everywhereml.data.collect import MjpegCollector


# Load the face detection model
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Open a video stream from the URL
url = "http://192.168.1.134"
video_stream = cv2.VideoCapture(url)

# Define the Wi-Fi sending function
def send_number_over_wifi():
    HOST = '192.168.1.134'  # Replace with the IP address of the device receiving the number
    PORT = 80  # Replace with the port number of the receiving device
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.connect((HOST, PORT)) #ip adres en poort invullen (udp of tcp vermoed ik)
        s.sendall(b'300')

# Loop through the frames in the video stream
while True:
    # Read the next frame from the video stream
    mjpeg_collector = MjpegCollector(address='http://192.168.1.134')
    print(dir(mjpeg_collector))
    frame_data_list = mjpeg_collector.collect_by_samples(5)
    frame_data = b"".join(frame_data_list)
    frame = cv2.imdecode(np.frombuffer(frame_data, dtype=np.uint8), cv2.IMREAD_COLOR)
    time.sleep(1)
    print(mjpeg_collector)
    if frame is None:
        continue
   
    
    
    # Apply the face detection model to the frame
    faces = face_cascade.detectMultiScale(frame, scaleFactor=1.1, minNeighbors=5)

    # Draw bounding boxes around the detected faces
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        # Send the number 300 over Wi-Fi
        send_number_over_wifi()

    # Display the output
    cv2.imshow('frame', frame)

    # Exit if the 'q' key is pressed
    if cv2.waitKey(1) == ord('q'):
        break

# Release the video stream and close the window
video_stream.release()
cv2.destroyAllWindows()


['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_collect', 'address', 'collect_by_duration', 'collect_by_samples', 'collect_many_classes', 'max_fps']


error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgcodecs\src\loadsave.cpp:798: error: (-215:Assertion failed) !buf.empty() in function 'cv::imdecode_'


In [19]:
import cv2
import urllib.request
import numpy as np

stream_url = 'http://192.168.1.134:81'  # Replace with your stream URL

# Open the video stream
stream = urllib.request.urlopen(stream_url)

# Create a VideoCapture object to read from the stream
video_capture = cv2.VideoCapture()

# Loop over the frames from the video stream
while True:
    # Read the next frame from the video stream
    ret, frame = video_capture.read()
    print(ret)
    # If there was an error reading the frame, break out of the loop
    if not ret:
        break

    # Display the frame
    cv2.imshow('Frame', frame)

    # Wait for a key press
    key = cv2.waitKey(1)

    # If the 'q' key was pressed, break out of the loop
    if key == ord('q'):
        break

# Release the resources
video_capture.release()
cv2.destroyAllWindows()


False


In [13]:
import cv2
import urllib.request
import requests
import numpy as np
import socket

# Load the face detection model
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Define the ESP32 Wrover's IP address and port number
url = 'http://192.168.0.126:81'

# Open the video stream from the ESP32 Wrover
stream = urllib.request.urlopen(url)

# Loop through the frames in the video stream
while True:
    # Read the next frame from the video stream
    buffer = bytes()
    while True:
        buffer += stream.read(2500)
        a = buffer.find(b'\xff\xd8')
        b = buffer.find(b'\xff\xd9')
        print(a, b)
        if a != -1 and b != -1:
            break
        elif a != -1 and b == -1:
            buffer = buffer[a:]
            break
    
    if a != -1 and b != -1 and b > a:
        #print(a,b)
        frame = cv2.imdecode(np.frombuffer(buffer[a:b+2], dtype=np.uint8), cv2.IMREAD_COLOR)

        if frame is not None:
            # Apply the face detection model to the frame
            faces = face_cascade.detectMultiScale(frame, scaleFactor=1.1, minNeighbors=5)

            # Draw bounding boxes around the detected faces
            for (x, y, w, h) in faces:
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                # Define the ESP32's IP address and port number
                ip = "192.168.0.126"
                port = 81

                # Define the integer to send
                message = 50

                # Create a socket object
                client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

                # Connect to the ESP32
                client_socket.connect((ip, port))

                # Send the integer as a string
                client_socket.send(str(message).encode())

                # Close the socket connection
                client_socket.close()
            # Display the output
            cv2.imshow('frame', frame)

    # Exit if the 'q' key is pressed
    if cv2.waitKey(1) == ord('q'):
        break
        
    # Remove the processed data from the buffer
    buffer = buffer[b+2:]


# Release the video stream and close the window
cv2.destroyAllWindows()


50 1484
722 658
1583 1519
768 690
1632 1561
818 754
0 1626
874 810
59 1669
927 863
112 48
972 908
155 75
1021 944
204 130
1070 995
252 188
1129 1065
328 256
1226 1141
425 339
1323 1227
522 430
1420 1316
619 521
1517 1406
716 603
1614 1472
813 666
12 1549
910 753
109 11
1007 835
206 39
1104 926
303 126
1201 1020
400 219
1298 1120
497 314
1395 1224
594 422
1492 1311
691 510
1589 1406
788 603
1686 1512
885 700
84 20
982 808
181 8
1079 908
278 90
1176 982
375 191
1273 1164
499 435
1491 1427
761 697
44 1767
1143 1079
450 386
1597 1533
918 830
239 127
1381 1235
702 537
23 1634
1165 959
486 280
1628 1428
949 765
270 115
1412 1243
733 571
54 1699
1196 1022
517 329
1659 1484
980 802
301 118
1443 1249
764 555
85 21
1227 1079
548 417
1690 1557
1011 846
332 148
1474 1235
795 562
116 4
1258 1032
579 377
1721 1519
1042 807
363 153
1505 1358
826 703
147 24
1289 1114
610 445
1752 1653
1076 1012
427 363
1659 1595
1024 923
389 193
1619 1304
984 615
349 7
1579 1236
944 596
309 96
1539 1201
904 562
269 56

1104 849
514 283
1834 1580
1244 989
654 398
64 0
1384 1068
794 470
204 53
1524 1119
934 514
344 20
1664 1218
1074 633
484 27
1804 1347
1214 768
624 167
34 1488
1354 891
764 300
174 23
1494 1028
904 433
314 25
1634 1162
1044 574
454 2
1774 1287
1184 697
594 106
4 1424
1324 840
734 246
144 38
1464 979
874 393
284 12
1604 1124
1014 532
424 4
1744 1266
1154 674
564 84
1884 1406
1294 821
704 230
114 8
1434 961
844 360
254 11
1574 1099
984 514
394 11
1714 1240
1124 650
534 54
1854 1381
1264 787
674 204
84 20
1404 922
814 332
224 73
1544 1063
954 474
364 40
1684 1201
1094 618
504 21
1824 1344
1234 755
644 170
54 1486
1374 897
784 309
194 43
1514 1041
924 452
334 10
1654 1177
1064 589
474 2
1794 1330
1204 728
614 138
24 1455
1344 867
754 282
164 13
1484 1010
894 406
304 15
1624 1149
1034 558
444 14
1764 1290
1174 688
584 108
1904 1420
1314 841
724 246
134 28
1454 970
864 382
274 2
1594 1128
1004 528
414 9
1734 1257
1144 665
554 69
1874 1396
1284 800
694 221
104 3
1424 957
834 357
244 1
1564 10

1324 855
734 251
144 38
1464 980
874 395
284 12
1604 1138
1014 540
424 19
1744 1268
1154 683
564 85
1884 1407
1294 818
704 229
114 8
1434 973
844 372
254 11
1574 1099
984 502
394 11
1714 1230
1124 653
534 57
1854 1372
1264 789
674 203
84 20
1404 934
814 343
224 73
1544 1069
954 481
364 40
1684 1215
1094 618
504 26
1824 1339
1234 760
644 162
54 1486
1374 901
784 309
194 43
1514 1036
924 448
334 10
1654 1177
1064 586
474 8
1794 1323
1204 725
614 136
24 1449
1344 873
754 277
164 13
1484 1006
894 420
304 15
1624 1150
1034 561
444 6
1764 1292
1174 694
584 108
1904 1423
1314 836
724 253
134 28
1454 975
864 385
274 2
1594 1118
1004 519
414 9
1734 1255
1144 665
554 81
1874 1390
1284 810
694 216
104 3
1424 950
834 359
244 1
1564 1081
974 498
384 1
1704 1224
1114 635
524 51
1844 1374
1254 771
664 186
74 10
1394 912
804 322
214 63
1534 1053
944 461
354 30
1674 1200
1084 606
494 11
1814 1339
1224 749
634 155
44 1474
1364 885
774 297
184 33
1504 1031
914 434
324 0
1644 1169
1054 576
464 26
1784 131

814 339
224 73
1544 1072
954 478
364 40
1684 1203
1094 613
504 21
1824 1346
1234 757
644 166
54 1491
1374 896
784 298
194 43
1514 1044
924 451
334 10
1654 1179
1064 590
474 25
1794 1322
1204 726
614 136
24 1463
1344 871
754 282
164 13
1484 1021
894 421
304 15
1624 1146
1034 559
444 39
1764 1295
1174 697
584 104
1904 1436
1314 842
724 243
134 28
1454 972
864 393
274 2
1594 1120
1004 520
414 9
1734 1260
1144 673
554 77
1874 1399
1284 796
694 214
104 3
1424 947
834 354
244 1
1564 1082
974 496
384 1
1704 1224
1114 635
524 55
1844 1370
1254 785
664 189
74 10
1394 924
804 331
214 63
1534 1051
944 471
354 30
1674 1192
1084 600
494 22
1814 1345
1224 749
634 162
44 1485
1364 885
774 304
184 33
1504 1027
914 443
324 0
1644 1159
1054 579
464 15
1784 1316
1194 725
604 119
14 1452
1334 865
744 273
154 3
1474 1009
884 431
294 5
1614 1165
1024 572
434 29
1754 1301
1164 716
574 120
1894 1441
1304 852
714 262
124 18
1444 989
854 402
264 8
1584 1130
994 542
404 21
1724 1255
1134 664
544 75
1864 1388
127

304 15
1624 1153
1034 554
444 8
1764 1292
1174 703
584 107
1904 1429
1314 843
724 248
134 28
1454 971
864 388
274 2
1594 1119
1004 522
414 9
1734 1254
1144 667
554 88
1874 1402
1284 809
694 212
104 3
1424 948
834 365
244 1
1564 1087
974 496
384 1
1704 1233
1114 640
524 43
1844 1365
1254 777
664 197
74 10
1394 915
804 326
214 63
1534 1051
944 470
354 30
1674 1192
1084 612
494 22
1814 1331
1224 755
634 157
44 1476
1364 881
774 290
184 33
1504 1021
914 435
324 0
1644 1169
1054 570
464 28
1784 1315
1194 720
604 129
14 1446
1334 860
744 259
154 3
1474 1000
884 407
294 5
1614 1135
1024 540
434 29
1754 1274
1164 687
574 100
1894 1418
1304 825
714 252
124 18
1444 961
854 378
264 8
1584 1109
994 520
404 21
1724 1247
1134 651
544 72
1864 1387
1274 806
684 203
94 30
1414 927
824 353
234 83
1554 1075
964 486
374 4
1694 1218
1104 624
514 32
1834 1355
1244 772
654 170
64 0
1384 906
794 317
204 53
1524 1049
934 455
344 20
1664 1189
1074 591
484 6
1804 1331
1214 731
624 150
34 1473
1354 881
764 287
17

384 1
1704 1219
1114 639
524 48
1844 1371
1254 774
664 193
74 10
1394 919
804 325
214 63
1534 1054
944 469
354 30
1674 1194
1084 605
494 19
1814 1337
1224 757
634 157
44 1484
1364 878
774 302
184 33
1504 1023
914 432
324 0
1644 1171
1054 574
464 28
1784 1310
1194 720
604 133
14 1453
1334 866
744 268
154 3
1474 997
884 415
294 5
1614 1138
1024 553
434 29
1754 1281
1164 693
574 102
1894 1421
1304 829
714 232
124 18
1444 963
854 371
264 8
1584 1110
994 524
404 21
1724 1252
1134 664
544 78
1864 1391
1274 802
684 209
94 30
1414 932
824 352
234 83
1554 1072
964 490
374 4
1694 1218
1104 626
514 32
1834 1356
1244 769
654 176
64 0
1384 904
794 319
204 53
1524 1054
934 459
344 20
1664 1202
1074 604
484 12
1804 1334
1214 731
624 155
34 1481
1354 892
764 306
174 23
1494 1051
904 454
314 25
1634 1193
1044 603
454 8
1774 1327
1184 731
594 154
4 1464
1324 878
734 289
144 38
1464 1013
874 419
284 12
1604 1137
1014 545
424 19
1744 1284
1154 677
564 86
1884 1402
1294 821
704 229
114 8
1434 955
844 366
2

KeyboardInterrupt: 